In [ ]:
!pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import emoji
import itertools
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, SimpleRNN, LSTM, Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.head()

,0,1
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [ ]:
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a very nice raise\t,2
3,she got me a nice present\t,2
4,ha ha ha it was so funny\t,2


Emojies from ` https://www.webfx.com/tools/emoji-cheat-sheet/


In [ ]:
emoji_dict = {
    0 : ":heart_suit:",
    1 : ":neutral_face:",
    2 : ":grinning_face:",
    3 : ":weary_face:",
    4 : ":fork_and_knife:"}

In [ ]:
for emoji_name, emoji_alias in emoji_dict.items():
    print(f"{emoji_name}: {emoji.emojize(emoji_alias)}")

0: ♥️
1: 😐
2: 😀
3: 😩
4: 🍴


In [ ]:
train.columns = ["text", "label"]
train.head()

,text,label
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [ ]:
test.columns = ["text", "label"]
test.head()

,text,label
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a very nice raise\t,2
3,she got me a nice present\t,2
4,ha ha ha it was so funny\t,2


#Preprocessing


In [ ]:
X_train = train["text"]
Y_train = train["label"]

X_test = test["text"]
Y_test = test["label"]
Y_eval = test["label"]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(132,) (132,) (56,) (56,)


In [ ]:
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()

Y_train = to_categorical(Y_train)

Y_test = to_categorical(Y_test)

<ipython-input-14-c87fba8346a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[ix] = X_train[ix].split()
<ipython-input-14-c87fba8346a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[ix] = X_train[ix].split()
<ipython-input-14-c87fba8346a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[ix] = X_train[ix].split()
<ipython-input-14-c87fba8346a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a c

In [ ]:
X_train[0]

['never', 'talk', 'to', 'me', 'again']

In [ ]:
X_train_first_sample = train['text'][0]
Y_train_first_sample = Y_train[0]

print(X_train_first_sample, Y_train_first_sample)

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [ ]:
sequence_lengths = np.array([len(seq) for seq in X_train])
unique_lengths, counts = np.unique(sequence_lengths, return_counts=True)
result = (unique_lengths, counts)
print(result)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4]))


In [ ]:
sequence_lengths_test = np.array([len(seq) for seq in X_test])
unique_lengths_test, counts_test = np.unique(sequence_lengths_test, return_counts=True)
result_test = (unique_lengths_test, counts_test)
print(result_test)


(array([2, 3, 4, 5, 6, 7, 8]), array([ 3, 13, 15, 14,  6,  3,  2]))


In [ ]:
embeddings_matrix = {}

f = open('./glove.6B.50d.txt', encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_matrix[word] = coefs
f.close()

In [ ]:
embeddings_matrix["i"].shape

(50,)

In [ ]:
from scipy import spatial
spatial.distance.cosine(embeddings_matrix["happy"], embeddings_matrix["fun"])

0.18571120500564575

In [ ]:
spatial.distance.cosine(embeddings_matrix["india"], embeddings_matrix["delhi"])

0.18572336435317993

In [ ]:
spatial.distance.cosine(embeddings_matrix["france"], embeddings_matrix["paris"])

0.19746702909469604

In [ ]:
X_train

0                 [never, talk, to, me, again]
1       [I, am, proud, of, your, achievements]
2      [It, is, the, worst, day, in, my, life]
3                        [Miss, you, so, much]
4                             [food, is, life]
                        ...                   
127          [he, had, to, make, a, home, run]
128                    [I, am, ordering, food]
129               [What, is, wrong, with, you]
130                             [I, love, you]
131                               [great, job]
Name: text, Length: 132, dtype: object

In [ ]:
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

# Function to get the embedding for a word, returning a zero vector if the word is not found
def get_embedding(word, embeddings_index, embedding_dim=50):
    return embeddings_index.get(word.lower(), np.zeros(embedding_dim))

# Populate embedding matrix for training data
for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        word = X_train[ix][ij]
        embedding_matrix_train[ix][ij] = get_embedding(word, embeddings_matrix)

# Populate embedding matrix for test data
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        word = X_test[ix][ij]
        embedding_matrix_test[ix][ij] = get_embedding(word, embeddings_matrix)


In [ ]:
model = Sequential()
model.add(LSTM(units= 64, input_shape = (10,50), return_sequences = True))
model.add(Dropout(0.3))
model.add(LSTM(units = 32))
model.add(Dropout(0.2))
model.add(Dense(units=10, activation = "relu"))
model.add(Dense(units= 5, activation = "softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            29440     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
Total params: 42241 (165.00 KB)
Trainable params: 42241 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
tf.config.run_functions_eagerly(True)
embedding_matrix_train_padded = pad_sequences(embedding_matrix_train, maxlen=10, padding='post')
embedding_matrix_test_padded = pad_sequences(embedding_matrix_test, maxlen=10, padding='post')


print("Shape of padded embedding_matrix_train:", embedding_matrix_train_padded.shape)
print("Shape of padded embedding_matrix_test:", embedding_matrix_test_padded.shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

hist = model.fit(embedding_matrix_train_padded,
                 Y_train,
                 epochs=40,
                 batch_size=10,
                 validation_data=(embedding_matrix_test_padded, Y_test),
                 validation_split = 0.2,
                 shuffle=True, callbacks=[early_stopping]

                )

Shape of padded embedding_matrix_train: (132, 10, 50)
Shape of padded embedding_matrix_test: (56, 10, 50)
Epoch 1/40


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


14/14 [==============================] - 3s 178ms/step - loss: 1.5919 - accuracy: 0.2652 - val_loss: 1.5583 - val_accuracy: 0.3036
Epoch 2/40
14/14 [==============================] - 4s 275ms/step - loss: 1.5442 - accuracy: 0.3182 - val_loss: 1.5236 - val_accuracy: 0.3214
Epoch 3/40
14/14 [==============================] - 7s 530ms/step - loss: 1.5250 - accuracy: 0.3485 - val_loss: 1.5071 - val_accuracy: 0.3750
Epoch 4/40
14/14 [==============================] - 4s 256ms/step - loss: 1.4770 - accuracy: 0.3712 - val_loss: 1.4734 - val_accuracy: 0.3571
Epoch 5/40
14/14 [==============================] - 2s 174ms/step - loss: 1.4083 - accuracy: 0.4470 - val_loss: 1.4132 - val_accuracy: 0.3393
Epoch 6/40
14/14 [==============================] - 2s 173ms/step - loss: 1.3286 - accuracy: 0.4848 - val_loss: 1.3874 - val_accuracy: 0.4286
Epoch 7/40
14/14 [==============================] - 3s 216ms/step - loss: 1.2626 - accuracy: 0.5152 - val_loss: 1.3134 - val_accuracy: 0.3571
Epoch 8/40
14/14 

In [ ]:
pred = np.argmax(model.predict(embedding_matrix_test_padded), axis=1)

2/2 [==============================] - 0s 57ms/step


In [ ]:
Y_test_indices = np.argmax(Y_test, axis=1)

accuracy = float(np.sum(pred == Y_test_indices)) / len(Y_test_indices)
print(f"Accuracy: {accuracy}")

Accuracy: 0.625


In [ ]:
emoji_dict

{0: ':heart_suit:',
 1: ':neutral_face:',
 2: ':grinning_face:',
 3: ':weary_face:',
 4: ':fork_and_knife:'}

In [ ]:
y_pred_classes = np.argmax(model.predict(embedding_matrix_test_padded), axis=1)


for t in range(len(test)):
    text = test["text"].iloc[t]

    predicted_label = int(y_pred_classes[t])
    actual_label = int(test["label"].iloc[t])

    predicted_emoji = emoji.emojize(emoji_dict[predicted_label], language='alias')
    actual_emoji = emoji.emojize(emoji_dict[actual_label], language='alias')

    print(f"Text: {text}")
    print(f"Prediction: {predicted_emoji}")
    print(f"Actual: {actual_emoji}")
    print()

2/2 [==============================] - 0s 45ms/step
Text: ['I', 'want', 'to', 'eat']
Prediction: 🍴
Actual: 🍴

Text: ['he', 'did', 'not', 'answer']
Prediction: 😩
Actual: 😩

Text: ['he', 'got', 'a', 'very', 'nice', 'raise']
Prediction: 😀
Actual: 😀

Text: ['she', 'got', 'me', 'a', 'nice', 'present']
Prediction: 😩
Actual: 😀

Text: ['ha', 'ha', 'ha', 'it', 'was', 'so', 'funny']
Prediction: 😀
Actual: 😀

Text: ['he', 'is', 'a', 'good', 'friend']
Prediction: 😀
Actual: 😀

Text: ['I', 'am', 'upset']
Prediction: 😩
Actual: 😩

Text: ['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight']
Prediction: 😐
Actual: 😀

Text: ['where', 'is', 'the', 'food']
Prediction: 😩
Actual: 🍴

Text: ['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha']
Prediction: 😀
Actual: 😀

Text: ['where', 'is', 'the', 'ball']
Prediction: 😐
Actual: 😐

Text: ['work', 'is', 'hard']
Prediction: 😩
Actual: 😩

Text: ['This', 'girl', 'is', 'messing', 'with', 'me']
Prediction: 😩
Actual: 😩

Text: ['are', 'you', 'serious']
Prediction: 😩
Actu